In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

# Tools for scaling data, PCA, and standard datasets
from sklearn import preprocessing, decomposition, datasets

# Tools for tracking learning curves and perform cross validation
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, validation_curve, learning_curve

# The k-NN learning algorithm
from sklearn.neighbors import KNeighborsClassifier as kNN

from sklearn.datasets import load_breast_cancer

# One-hot enconding and ordinal encoding
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder

# Apply transformations to columns
from sklearn.compose import make_column_transformer

In [ ]:
pima = pd.read_csv("Datasets/diabetes.csv")
pima.info()

In [ ]:
pima.head()

The `Outcome` column contains the labels. We use this to construct our sets of training points and training labels.

In [ ]:
X = pima.drop(columns='Outcome').values
y = pima['Outcome'].values

As before, we count the proportions of positive and negative labels.

In [ ]:
np.unique(y, return_counts=True)

Then we split the dataset in training set (60%) and test set (40%) using stratification.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.4,random_state=42, stratify=y)

We plot the cross-validated estimates of the risk for different values of $k$.

In [ ]:
neighbors = range(1,200,20)
train_score, val_score = validation_curve(kNN(), X, y, param_name='n_neighbors', param_range=neighbors, cv=5)

Once more, the regions of underfitting and overfitting for the parameter $k$ are clearly seen in the plot.

In [ ]:
plt.title('k-NN vs. number of neighbors')
plt.plot(neighbors, np.mean(val_score, 1), label='Testing accuracy')
plt.plot(neighbors, np.mean(train_score, 1), label='Training accuracy')
plt.legend()
plt.xlabel('Number of neighbors')
plt.ylabel('Accuracy')
plt.show()

### Cross-validation to evaluate performance of a given algorithm
The function `cross_val_score()` performs cross validation to estimate the risk of the classifier output by a given algorithm.

Here is an example using $5$-fold cross-validation on the entire dataset to evaluate the performance of $21$-NN.

In [ ]:
knn = kNN(n_neighbors=21)
scores = cross_val_score(knn, X, y, cv=20)
scores, scores.mean()

### Grid-search to find best value of parameter for the learning algorithm
We can use the function `GridSearch()` to look for the best parameter of an algorithm using the entire dataset.
- Repeat 5-fold cross-validation on the entire dataset for each value of the parameter in the grid
- Select the parameter with the best cross-validated score

In [ ]:
k_grid = {'n_neighbors': range(1, 100, 20)}
learner = GridSearchCV(estimator=kNN(), param_grid=k_grid, cv=20, return_train_score=True)
learner.fit(X, y)
learner.best_params_, learner.best_score_ # vars containing the best parameter value and its corresponding cv score

The algorithm with the best parameter, $21$-NN, is available in the variable `learner.best_estimator_`, that is `learner.best_estimator_ = kNN(n_neighbors=21)`

We repeat the evaluation of this algorithm using 5-fold cross-validation.

In [ ]:
model = learner.best_estimator_
scores = cross_val_score(model, X, y, cv=20)
scores.mean()

### Nested cross-validation to evaluate performance of a learning algorithm with parameters to tune
We saw that cross-validation allows us to use the data for choosing a good value of the parameter. However, we are still left with the problem of estimating the risk of the classifier generated by the algorithm. Nested cross-validation provides a way of estimating the risk of a classifier generated by an algorithm whose parameters are tuned using cross-validation on the training set.

In the following example, we:
- Run 5-fold cross-validation on the entire dataset
- On the training part of each fold, run *internal* 5-fold cross-validation to find the best value of the parameter
- Re-train the model on the training part of the outer fold using the optimized parameter
- Test the model on the testing part of the outer fold.

In [ ]:
k_grid = {'n_neighbors': range(1,100,20)}
learner = GridSearchCV(estimator=kNN(), param_grid=k_grid, cv=20) # internal C-V
scores = cross_val_score(learner, X, y, cv=20) # external C-V
scores, scores.mean()

Note that the nested cross-validated estimate is $0.72$, while the cross-validated estimate we computed above using grid search on the entire dataset is higher, $0.74$. This discrepancy occurs because the nested CV estimate is statistically more accurate than the cross-validated estimate, which tends to overfit a bit.

## Preprocessing the dataset
Many learning algorithms may work better when the training set is rescaled in certain ways. Note that, in order to avoid contributing to overfitting, these rescalings should not depend on the training examples.

We illustrate the most popular rescaling technique on the cancer dataset.

In [ ]:
data = load_breast_cancer()
X = data.data
y = data.target

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.4,random_state=42,stratify=y)

The `StandardScaler()` function standardizes the values of each feature $i$. If $x_1(i),\ldots,x_m(i)$ are the values of the $i$-th feature in the dataset $x(1),\dots,x(m)$, then `StandardScaler()` replaces each value $x_t(i)$ with
$$x_t(i)' = \frac{x_t(i)-\mu_i}{\sigma_i}$$
where $$\mu_i = \frac{1}{m}\sum_{t=1}^m x_t(i) \;\;\;\textrm{and}\;\;\; \sigma_i^2 = \frac{1}{m}\sum_{t=1}^m \bigl(x_t(i)-\mu_i\big)^2$$

Note that `standard_scaler.fit_transform()` is used to compute $\mu_i$ and $\sigma_i$ for each feature $i$ on the training data and then to rescale the training data. The testing data are rescaled using the parameters computed on the training data. Allowing the learner to compute the rescaling parameters using the testing data would imply that the test set is made available (without labels) before the classifier is generated. This is typically not allowed in the statistical learning model.

In [ ]:
standard_scaler = preprocessing.StandardScaler()
X_train_standard = standard_scaler.fit_transform(X_train)
X_test_standard = standard_scaler.transform(X_test)

Next, we compute the test set performance with and without rescaling for different values of $k$.

In [ ]:
neighbors = range(1,8)
test_scores = []
test_scores_standard = []

for k in neighbors:
    knn = kNN(n_neighbors=k)
    knn.fit(X_train, y_train)
    test_scores.append(knn.score(X_test, y_test))
    knn.fit(X_train_standard, y_train)
    test_scores_standard.append(knn.score(X_test_standard, y_test))

Plotting the perfomance in both cases shows the benefits of rescaling.

In [ ]:
plt.title('k-NN vs. number of neighbors')
plt.plot(neighbors, test_scores, label='Testing accuracy')
plt.plot(neighbors, test_scores_standard, label='Testing accuracy (scaled)')
plt.legend()
plt.xlabel('Number of neighbors')
plt.ylabel('Accuracy')
plt.show()

We now repeat the same exercise use the Pima Indians dataset.

In [ ]:
X = pima.drop(columns='Outcome').values
y = pima['Outcome'].values
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.4,random_state=42, stratify=y)
standard_scaler = preprocessing.StandardScaler()
X_train_standard = standard_scaler.fit_transform(X_train)
X_test_standard = standard_scaler.transform(X_test)

In [ ]:
neighbors = range(1,100,20)
test_scores = []
test_scores_standard = []

for k in neighbors:
    knn = kNN(n_neighbors=k)
    knn.fit(X_train, y_train)
    test_scores.append(knn.score(X_test, y_test))
    knn.fit(X_train_standard, y_train)
    test_scores_standard.append(knn.score(X_test_standard, y_test))

Also in this case, we see that rescaling helps boost the test accuracy when $k$ is not chosen optimally.

In [ ]:
plt.title('k-NN vs. number of neighbors')
plt.plot(neighbors, test_scores, label='Testing accuracy')
plt.plot(neighbors, test_scores_standard, label='Testing accuracy (scaled)')
plt.legend()
plt.xlabel('Number of neighbors')
plt.ylabel('Accuracy')
plt.show()

## Feature encoding
Scikit_learn does not work on datasets with categorical features. Here we show how to transform categorical values into numerical values.

In [ ]:
data = pd.read_csv("Datasets/StudentsPerformance.csv")
data.head()

In [ ]:
data.info()

This dataset has 5 categorical features. The labels to predict are `math score`, `reading score`, and `writing score`.

In [ ]:
score = data[['math score', 'reading score', 'writing score']]
score.head()

To have a single label to predict, we replace the three `score` labels with their mean.

In [ ]:
# Calculate average of all test results
data["mean score"] = data[['math score','reading score','writing score']].mean(axis = 'rows')

# Drop math score, reading score and writing score
data = data.drop(columns=['math score', 'reading score', 'writing score'])

# Show first 5 rows of new dataframe
data.head()

If the values of a categorical features are not ranked, then we use **one-hot enconding**. This corresponds to creating a new dummy binary feature for each value in the range of the categorical feature. For instance, for `gender` which takes values in `{'male', 'female'}` we create two dummy features, say `gender_m` and `gender_f`.

Then, every occurrence of `gender = 'male'` is replaced by `gender_m = 1` and `gender_f = 0`. Vice versa, every occurrence of `gender = 'female'` is replace by `gender_m = 0` and `gender_f = 1`.

Note that one-hot encoding requires a number of dummy variables equal to the cardinality of the range of the categorical feature.

In [ ]:
# Instantiate OneHotEncoder

ohe = OneHotEncoder(sparse_output = False)

In [ ]:
# Apply OneHotEncoder to the gender column 

ohe.fit_transform(data[['gender']])

In [ ]:
#Check the result

ohe.fit_transform(data[['gender']])[:5]

In [ ]:
# The first 5 rows of the gender column for comparison

data['gender'].head()

If the values of the categorical features are ranked according to some criterion, then one should use **ordinal encoding** instead of one-hot encoding.

In [ ]:
# Unique values in the parental level of education column

data['parental level of education'].unique()

The values of `parental level of education` represent degrees of education, which have a natural ranking.

Ordinal encoding maps values to positive integers `1,2,...` starting from the lowest position in the ranking.

In [ ]:
# Specify the order for the level of education 

education_categories = ['some high school', 'high school', 'some college', "associate's degree", "bachelor's degree", "master's degree"]

In [ ]:
# Instantiate ordinal encoder

oe = OrdinalEncoder(categories = [education_categories])

In [ ]:
# Apply ordinal encoder to parental level of education column

oe.fit_transform(data[['parental level of education']])

Create matrix with datapoints and vector of labels.

In [ ]:
X = data.drop(columns='mean score')
y = data['mean score']

We now convert all categorical variable at once using the function `make_column_transformer`. We use the object `ohe` of type one-hot encoder for all columns but `parental level of education` for which we use the object `oe` of type ordinal encoder.

In [ ]:
column_transform = make_column_transformer(
    (ohe, ['gender', 'race/ethnicity', 'lunch', 'test preparation course']), 
    (oe, ['parental level of education']))

In [ ]:
X = column_transform.fit_transform(X)

X.shape

Note that the number of features has increased from 5 to 12.

Here is the resulting dataset which now contains only numerical features.

In [ ]:
X